In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
import multiprocessing
from pyspark.sql.functions import asc, desc, col, sum, mean, concat, lit, regexp_extract, regexp_replace, when, max, min

Create a jupyter notebook or python script named spark101 for this exercise.

### 1. Create a spark data frame that contains your favorite programming languages.
- The name of the column should be language


In [2]:
pandas_dataframe = pd.DataFrame({
    "Python": np.random.choice([0,1], 20),
    "C++": np.random.choice([0,1], 20),
    "Javascript": np.random.choice([0,1], 20),
})

spark = pyspark.sql.SparkSession.builder.getOrCreate()
df = spark.createDataFrame(pandas_dataframe)

- View the schema of the dataframe

In [3]:
df.schema

StructType(List(StructField(Python,LongType,true),StructField(C++,LongType,true),StructField(Javascript,LongType,true)))

- Output the shape of the dataframe

In [4]:
df.toPandas().shape

(20, 3)

- Show the first 5 records in the dataframe

In [5]:
df.show(5)

+------+---+----------+
|Python|C++|Javascript|
+------+---+----------+
|     0|  1|         1|
|     1|  0|         1|
|     1|  0|         1|
|     1|  1|         1|
|     0|  1|         1|
+------+---+----------+
only showing top 5 rows



### 2. Load the mpg dataset as a spark dataframe.

In [6]:
from pydataset import data

In [7]:
mpg = data('mpg')

In [8]:
mpg.head()

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact


In [9]:
df = spark.createDataFrame(mpg)
df.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|
|        audi|                a4|  3.1|2008|  6|  auto(av)|  f| 18| 27|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|manual(m5)|  4| 18| 26|  p|compact|
|        audi|        a4 quattro|  1.8|1999|  4|  auto(l5)|  4| 16| 25|  p|c

- a. Create 1 column of output that contains a message like the one below:
        The 1999 audi a4 has a 4 cylinder engine.

In [10]:
df.select(concat(lit('The '), df.year, lit(' '), df.manufacturer, lit(
    ' '), df.model, lit(' has a '), df.cyl, lit(' cylinder engine'))).show(truncate=False)

+-----------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine)|
+-----------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 2008 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 2008 audi a4 quattro has a 4 cylind

- For each vehicle.

- b. Transform the trans column so that it only contains either manual or auto.

In [11]:
df.select('trans').show()

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
|manual(m5)|
|  auto(av)|
|manual(m5)|
|  auto(l5)|
|manual(m6)|
|  auto(s6)|
|  auto(l5)|
|manual(m5)|
|  auto(s6)|
|manual(m6)|
|  auto(l5)|
|  auto(s6)|
|  auto(s6)|
|  auto(l4)|
|  auto(l4)|
+----------+
only showing top 20 rows



In [12]:
df.where(col('trans').like("%auto%")).select(df.trans).show()

+--------+
|   trans|
+--------+
|auto(l5)|
|auto(av)|
|auto(l5)|
|auto(av)|
|auto(l5)|
|auto(s6)|
|auto(l5)|
|auto(s6)|
|auto(l5)|
|auto(s6)|
|auto(s6)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(l4)|
|auto(s6)|
|auto(l4)|
|auto(l4)|
+--------+
only showing top 20 rows



In [13]:
df.select(df.trans,
         when(df.trans.like('auto%'), 'auto')
         .otherwise('manual')
         .alias('transmission')
         ).show()

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(av)|        auto|
|manual(m5)|      manual|
|  auto(l5)|        auto|
|manual(m6)|      manual|
|  auto(s6)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(s6)|        auto|
|manual(m6)|      manual|
|  auto(l5)|        auto|
|  auto(s6)|        auto|
|  auto(s6)|        auto|
|  auto(l4)|        auto|
|  auto(l4)|        auto|
+----------+------------+
only showing top 20 rows



### 3. Load the tips dataset as a spark dataframe.

In [14]:
tips = data('tips')
df = spark.createDataFrame(tips)
df.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



- a. What percentage of observations are smokers?

In [15]:
df.toPandas().shape

(244, 7)

In [16]:
(df.where(df.smoker == 'Yes').count()) / (df.toPandas().shape[0]) * 100

38.114754098360656

- b. Create a column that contains the tip percentage

In [17]:
df.select(
    (df.tip / df.total_bill)
    .alias('tip_percentage')
).show()

+-------------------+
|     tip_percentage|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
| 0.1665043816942551|
|0.14180374361883155|
|0.10181582360570687|
|0.16277807921866522|
|0.20364126770060686|
|0.18164967562557924|
| 0.1616650532429816|
|0.22774708410067526|
|0.20624631703005306|
|0.16222760290556903|
+-------------------+
only showing top 20 rows



- c. Calculate the average tip percentage for each combination of sex and smoker.

In [18]:
col1 = df.select((df.tip / df.total_bill).alias('tip_percentage'))

In [19]:
col1.show()

+-------------------+
|     tip_percentage|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
|0.18623962040332148|
|0.22805017103762829|
|0.11607142857142858|
|0.13031914893617022|
| 0.2185385656292287|
| 0.1665043816942551|
|0.14180374361883155|
|0.10181582360570687|
|0.16277807921866522|
|0.20364126770060686|
|0.18164967562557924|
| 0.1616650532429816|
|0.22774708410067526|
|0.20624631703005306|
|0.16222760290556903|
+-------------------+
only showing top 20 rows



In [20]:
df.groupBy('sex','smoker').agg(mean((df.tip / df.total_bill).alias('tip_percentage'))).show()

+------+------+-------------------------------------------+
|   sex|smoker|avg((tip / total_bill) AS `tip_percentage`)|
+------+------+-------------------------------------------+
|  Male|    No|                         0.1606687151291298|
|  Male|   Yes|                         0.1527711752024851|
|Female|    No|                         0.1569209707691836|
|Female|   Yes|                        0.18215035269941035|
+------+------+-------------------------------------------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [21]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



- Convert the temperatures to farenheight.
celsius * 1.8 = fahrenheit - 32

In [22]:
def fahrenheit(celsius):
    f = celsius * 1.8 + 32
    return f    

In [23]:
weather.select('*',fahrenheit(weather.temp_max).alias('temp_max_f'),fahrenheit(weather.temp_min).alias('temp_min_f')).show()

+----------+-------------+--------+--------+----+-------+------------------+------------------+
|      date|precipitation|temp_max|temp_min|wind|weather|        temp_max_f|        temp_min_f|
+----------+-------------+--------+--------+----+-------+------------------+------------------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|55.040000000000006|              41.0|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|             51.08|             37.04|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|             53.06|             44.96|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|             53.96|             42.08|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|48.019999999999996|             37.04|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|             39.92|             35.96|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|             44.96|             37.04|
|2012-01-08|          0.0|    10.0|     

- Which month has the most rain, on average?

In [24]:
from pyspark.sql.functions import month, year, quarter

In [25]:
rain = weather.where(weather['weather'] == 'rain')
rain.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-21|          3.0|     8.3|     3.3| 8.2|   rain|
|2012-01-22|          6.1|     6.7|     2.2| 4.8|   rain|
|2012-01-23|          0.0|     8.3|     1.1| 3.6|   rain|
|2012-01-24|          8.6|    10.0|     2.2| 5.1|   rain|
|2012-01-25|          8.1|     8.9|     4.4| 5.4|   rain|
|2012-01-26|          4.8|     8.9|     1.1| 4.8|   rain|
|2012-01-28|  

In [26]:
(
    rain.withColumn("month", month("date"))
    .groupBy("month")
    .agg(mean("precipitation").alias("total_rainfall"))
    .sort(desc('total_rainfall'))
    .show()
)

+-----+-------------------+
|month|     total_rainfall|
+-----+-------------------+
|   10|              9.675|
|   11|               8.42|
|    1|  6.614285714285714|
|    8|  6.433333333333334|
|   12| 5.0260869565217385|
|    3|  4.921621621621622|
|    6|  3.952631578947368|
|    4|  3.429999999999999|
|    5| 3.2624999999999997|
|    2| 3.1725000000000003|
|    7| 1.8785714285714286|
|    9|0.22499999999999998|
+-----+-------------------+



- Which year was the windiest?

In [27]:
(
   weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_wind"))
    .sort(desc('total_wind'))
    .show()
)

+----+------------------+
|year|        total_wind|
+----+------------------+
|2012|1244.6999999999998|
|2014|1236.5000000000005|
|2015|            1153.3|
|2013|1100.8000000000002|
+----+------------------+



- What is the most frequent type of weather in January?

In [28]:
jan = weather.where(month(weather.date) == '01')
jan.show()

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|
|2012-01-13|          0.0|     5.0|    -2.8| 1.3|    sun|
|2012-01-14|          4.1|     4.4|     0.6| 5.3|   snow|
|2012-01-15|  

In [29]:
jan.count(), weather.count()

(124, 1461)

In [30]:
(
    jan.withColumn('month', month('date'))
    .groupBy(jan.weather.alias('January_weather'))
    .count()
    .sort(desc('count'))
    .show()
)

+---------------+-----+
|January_weather|count|
+---------------+-----+
|            fog|   38|
|           rain|   35|
|            sun|   33|
|        drizzle|   10|
|           snow|    8|
+---------------+-----+



- What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [31]:
weather = weather.withColumn('quarter',quarter('date')).withColumn('year',year('date')).withColumn('month',month('date'))
weather.show()

+----------+-------------+--------+--------+----+-------+-------+----+-----+
|      date|precipitation|temp_max|temp_min|wind|weather|quarter|year|month|
+----------+-------------+--------+--------+----+-------+-------+----+-----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|      1|2012|    1|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|      1|2012|    1|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|      1|2012|    1|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|      1|2012|    1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|      1|2012|    1|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|      1|2012|    1|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|      1|2012|    1|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|      1|2012|    1|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|      1|2012|    1|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|      1|2012|    1|

In [32]:
sunny_jul = weather.where(month(weather.date) == '07').where(weather.weather == 'sun').where((year(weather.date) == '2013') | (year(weather.date) == '2014'))
sunny_jul.show()

+----------+-------------+--------+--------+----+-------+-------+----+-----+
|      date|precipitation|temp_max|temp_min|wind|weather|quarter|year|month|
+----------+-------------+--------+--------+----+-------+-------+----+-----+
|2013-07-01|          0.0|    31.7|    18.3| 2.3|    sun|      3|2013|    7|
|2013-07-02|          0.0|    28.3|    15.6| 3.0|    sun|      3|2013|    7|
|2013-07-03|          0.0|    26.1|    16.7| 3.2|    sun|      3|2013|    7|
|2013-07-05|          0.0|    23.3|    13.9| 2.6|    sun|      3|2013|    7|
|2013-07-06|          0.0|    26.1|    13.3| 2.2|    sun|      3|2013|    7|
|2013-07-07|          0.0|    23.9|    13.9| 2.9|    sun|      3|2013|    7|
|2013-07-08|          0.0|    26.7|    13.3| 2.8|    sun|      3|2013|    7|
|2013-07-09|          0.0|    30.0|    15.0| 2.5|    sun|      3|2013|    7|
|2013-07-10|          0.0|    22.2|    13.9| 2.6|    sun|      3|2013|    7|
|2013-07-11|          0.0|    22.8|    12.2| 3.0|    sun|      3|2013|    7|

In [33]:
sunny_jul.groupBy('month').agg(mean('temp_min'),mean('temp_max')).show()

+-----+-----------------+------------------+
|month|    avg(temp_min)|     avg(temp_max)|
+-----+-----------------+------------------+
|    7|14.18269230769231|26.828846153846158|
+-----+-----------------+------------------+



- What percentage of days were rainy in q3 of 2015?

In [34]:
q3_2015 = weather.where((weather.quarter == 3) & (weather.year == '2015'))
q3_2015.show()

+----------+-------------+--------+--------+----+-------+-------+----+-----+
|      date|precipitation|temp_max|temp_min|wind|weather|quarter|year|month|
+----------+-------------+--------+--------+----+-------+-------+----+-----+
|2015-07-01|          0.0|    32.2|    17.2| 4.3|    sun|      3|2015|    7|
|2015-07-02|          0.0|    33.9|    17.8| 3.4|    sun|      3|2015|    7|
|2015-07-03|          0.0|    33.3|    17.8| 2.6|    sun|      3|2015|    7|
|2015-07-04|          0.0|    33.3|    15.0| 2.9|    sun|      3|2015|    7|
|2015-07-05|          0.0|    32.8|    16.7| 2.1|    sun|      3|2015|    7|
|2015-07-06|          0.0|    29.4|    15.6| 3.2|drizzle|      3|2015|    7|
|2015-07-07|          0.0|    27.2|    13.9| 2.4|    sun|      3|2015|    7|
|2015-07-08|          0.0|    30.0|    14.4| 1.9|drizzle|      3|2015|    7|
|2015-07-09|          0.0|    28.9|    14.4| 3.4|    sun|      3|2015|    7|
|2015-07-10|          0.0|    21.1|    16.7| 3.7|    sun|      3|2015|    7|

In [35]:
q3_2015.count()

92

In [36]:
(q3_2015.where(q3_2015.weather == 'rain').count()) / q3_2015.count() * 100

2.1739130434782608

- For each year, find what percentage of days it rained (had non-zero precipitation).

In [37]:
from pyspark.sql.functions import expr

In [38]:
weather.groupBy('year').count().show()

+----+-----+
|year|count|
+----+-----+
|2015|  365|
|2013|  365|
|2014|  365|
|2012|  366|
+----+-----+



In [39]:
year_list = list(range(2012,2016))
year_list

[2012, 2013, 2014, 2015]

In [40]:
weather.dtypes

[('date', 'string'),
 ('precipitation', 'double'),
 ('temp_max', 'double'),
 ('temp_min', 'double'),
 ('wind', 'double'),
 ('weather', 'string'),
 ('quarter', 'int'),
 ('year', 'int'),
 ('month', 'int')]

In [41]:
weather.where((weather.precipitation > 0) & (weather.year == 2015)).count() / weather.where(weather.year == 2015).count()

0.39452054794520547

In [42]:
def pct_rainy_days(year):
    pct = round(weather.where((weather.precipitation > 0) & (weather.year == year)).count() / weather.where(weather.year == year).count(),2)
    print('\npercentage of rainy days in ',year,': ',pct) 

In [43]:
[pct_rainy_days(year) for year in year_list]


percentage of rainy days in  2012 :  0.48

percentage of rainy days in  2013 :  0.42

percentage of rainy days in  2014 :  0.41

percentage of rainy days in  2015 :  0.39


[None, None, None, None]

In [44]:
weather_df = weather.toPandas()

In [45]:
weather_df[(weather_df.precipitation > 0) & (weather_df.year == 2012)]

,date,precipitation,temp_max,temp_min,wind,weather,quarter,year,month
1,2012-01-02,10.9,10.6,2.8,4.5,rain,1,2012,1
2,2012-01-03,0.8,11.7,7.2,2.3,rain,1,2012,1
3,2012-01-04,20.3,12.2,5.6,4.7,rain,1,2012,1
4,2012-01-05,1.3,8.9,2.8,6.1,rain,1,2012,1
5,2012-01-06,2.5,4.4,2.2,2.2,rain,1,2012,1
...,...,...,...,...,...,...,...,...,...
358,2012-12-24,0.3,5.6,2.8,2.8,rain,4,2012,12
359,2012-12-25,13.5,5.6,2.8,4.2,snow,4,2012,12
360,2012-12-26,4.6,6.7,3.3,4.9,rain,4,2012,12
361,2012-12-27,4.1,7.8,3.3,3.2,rain,4,2012,12


In [46]:
weather_df[weather_df.year == 2012]

,date,precipitation,temp_max,temp_min,wind,weather,quarter,year,month
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle,1,2012,1
1,2012-01-02,10.9,10.6,2.8,4.5,rain,1,2012,1
2,2012-01-03,0.8,11.7,7.2,2.3,rain,1,2012,1
3,2012-01-04,20.3,12.2,5.6,4.7,rain,1,2012,1
4,2012-01-05,1.3,8.9,2.8,6.1,rain,1,2012,1
...,...,...,...,...,...,...,...,...,...
361,2012-12-27,4.1,7.8,3.3,3.2,rain,4,2012,12
362,2012-12-28,0.0,8.3,3.9,1.7,rain,4,2012,12
363,2012-12-29,1.5,5.0,3.3,1.7,rain,4,2012,12
364,2012-12-30,0.0,4.4,0.0,1.8,drizzle,4,2012,12
